# Assignment 1
## Digital Musicology

@authors: Joris Monnet, Xingyu Pan, Yutaka Osaki, Yiwei Liu

Due Date: 24/04/2024

In [1]:
import music21
from music21 import midi, note, stream , interval, duration , converter

### Duration of notes
#### 1. Modify the note timing of a specific beat within a specific measure(s) of a music stream(s).

In [2]:
# chage duration of the beats
def change_duration_specific_beats_in_stream(s, start_measure, end_measure, target_beats, duration_factor):
    # target_beats should be a list of beats where duration needs to be changed, e.g., [1, 3]
    for measure_number in range(start_measure, end_measure + 1):
        measure = s.measure(measure_number)
        if measure is not None:
            for n in measure.notes:  # Iterating over notes directly
                if n.beat in target_beats:  # Check if the note's beat is in the list of target beats
                    n.duration.quarterLength *= duration_factor
    return s

#### 2. Adjusts the note duration of one track within a given measure, and automatically adjusts the note duration of the other track to ensure that the total duration of the two tracks remains the same.

### Pitches of notes 
#### 1. Extract the specific notes and adjust their pitches

In [3]:
# extract the notes and adjust their pitches
def adjust_note_in_measures(s, start_measure, end_measure, note_index, pitch_interval):
    """
    Adjusts the pitch of a specific note in a given range of measures.

    Parameters:
        s (music21.stream.Stream): The music stream to modify.
        start_measure (int): The starting measure number.
        end_measure (int): The ending measure number.
        note_index (int): The index of the note in its measure to adjust (0-based index).
        pitch_interval (int): The number of semitones to transpose the note (positive or negative).

    Returns:
        music21.stream.Stream: The modified music stream.
    """
    for measure_number in range(start_measure, end_measure + 1):
        measure = s.measure(measure_number)  # Get the specific measure
        if measure:
            notes = [n for n in measure.notes]  # Get all notes in the measure
            if len(notes) > note_index:  # Check if the note index is within the range of available notes
                target_note = notes[note_index]
                target_note.transpose(interval.ChromaticInterval(pitch_interval), inPlace=True)
    return s

#### 2. Detailed finding: Specify the note with the highest pitch in the measure and increase its intensity.

In [4]:
def accentuate_highest_note_in_measure(s, measure_number, accent_factor):
    """
    Increases the velocity of all notes with the highest pitch in the specified measure.

    :param s: music21 stream object
    :param measure_number: the measure number to find and accentuate the highest notes
    :param accent_factor: the factor by which to increase the velocity (e.g., 1.2 for 20% increase)
    :return: modified music21 stream
    """
    measure = s.measure(measure_number)
    if measure is None:
        print(f"No measure found with the number {measure_number}.")
        return s

    highest_pitch = None
    notes_to_accentuate = []

    # Ensure to only handle note and chord objects
    for element in measure.recurse().notesAndRests:
        if hasattr(element, 'isNote') and element.isNote:
            if highest_pitch is None or element.pitch.midi > highest_pitch:
                highest_pitch = element.pitch.midi
                notes_to_accentuate = [element]
            elif element.pitch.midi == highest_pitch:
                notes_to_accentuate.append(element)
        elif hasattr(element, 'isChord') and element.isChord:
            for p in element.pitches:
                if highest_pitch is None or p.midi > highest_pitch:
                    highest_pitch = p.midi
                    notes_to_accentuate = [element]
                elif p.midi == highest_pitch:
                    notes_to_accentuate.append(element)

    # Increase the velocity of all notes/chords with the highest pitch
    for n in notes_to_accentuate:
        if hasattr(n, 'isNote') and n.isNote:
            n.volume.velocity = min(max(int(n.volume.velocity * accent_factor), 0), 127)
        elif hasattr(n, 'isChord') and n.isChord:
            for note in n.notes:
                note.volume.velocity = min(max(int(note.volume.velocity * accent_factor), 0), 127)

    return s

#### 2. Detailed finding: Increases the volume of the highest-pitched note in triples of thirty-second notes within specified measures.

In [5]:
def increase_volume_of_highest_note_in_triples(score, start_measure_number, end_measure_number, track_number=0, volume_increase=10):
    """
    Increases the volume of the highest-pitched note in triples of thirty-second notes within specified measures
    in a specific track of a score.
    
    Params:
        score (music21.stream.Score): The Score object to be processed.
        track_number (int): The track number to process (0-indexed).
        start_measure_number (int): The measure number to start processing (inclusive).
        end_measure_number (int): The measure number to end processing (inclusive).
        volume_increase (int): The amount by which to increase the volume of the highest-pitched note.
    
    Returns:
        music21.stream.Score: The modified Score object with increased volumes for the highest-pitched notes in the specified measures of the specified track.
    """
    
    target_part = score.parts[track_number]
    
    for i in range(start_measure_number, end_measure_number + 1):
        target_measure = target_part.measure(i)
        notes = target_measure.notes
        # Iterate through each triple group of notes in the measure
        for j in range(len(notes) - 2):
            note1, note2, note3 = notes[j], notes[j+1], notes[j+2]
            if note1.duration.quarterLength == 0.125 and note2.duration.quarterLength == 0.125 and note3.duration.quarterLength == 0.125:
                # Determine the highest note
                pitches = [note1.pitch.midi, note2.pitch.midi, note3.pitch.midi]
                max_pitch = max(pitches)
                highest_note = None
                if note1.pitch.midi == max_pitch:
                    highest_note = note1
                elif note2.pitch.midi == max_pitch:
                    highest_note = note2
                else:
                    highest_note = note3
                
                # Increase the volume of the highest note
                highest_note.volume.velocity = min(highest_note.volume.velocity + volume_increase, 127)
            
        print(f"Measure {i} adjusted.")

    return score

#### 3.  Increases the volume of the higher-pitched note in pairs of sixteenth notes within specified measures

In [6]:
def increase_volume_of_higher_notes_in_track(score, start_measure_number, end_measure_number, track_number=0, volume_increase=10):
    """
    Increases the volume of the higher-pitched note in pairs of sixteenth notes within specified measures in a specific track of a score.
    
    Params:
        score (music21.stream.Score): The Score object to be processed.
        track_number (int): The track number to process (1-indexed).
        start_measure_number (int): The measure number to start processing (inclusive).
        end_measure_number (int): The measure number to end processing (inclusive).
        volume_increase (int): The amount by which to increase the volume of the higher-pitched note.

    Returns:
        music21.stream.Score: The modified Score object with increased volumes for higher-pitched notes in the specified measures of the specified track.
    """
    
    target_part = score.parts[track_number]
    
    for i in range(start_measure_number, end_measure_number + 1):
        target_measure = target_part.measure(i)
        for note1, note2 in zip(target_measure.notes[:-1], target_measure.notes[1:]):
            if note1.duration.quarterLength == 0.25 and note2.duration.quarterLength == 0.25:
                # Extract pitches; handle both Note and Chord objects
                pitch1 = note1.pitches[-1] if isinstance(note1, music21.chord.Chord) else note1.pitch
                pitch2 = note2.pitches[-1] if isinstance(note2, music21.chord.Chord) else note2.pitch

                # Increase volume of the higher-pitched note
                if pitch1 > pitch2:
                    note1.volume.velocity += volume_increase
                else:
                    note2.volume.velocity += volume_increase
            
        print(f"Measure {i} adjusted.")

    return score

### Dynamics
#### 1. Modify the velocity（adjust the note's velocity one by one）

In [7]:
def change_velocity_measures_in_stream(s, start_measure, end_measure, velocity_factor):
    for measure_number in range(start_measure, end_measure + 1):
        measure = s.measure(measure_number)  
        if measure is not None:
            # modify intensity
            for n in measure.recurse().notes:
                # Calculate the new tone intensity and make sure it is in the range of the MIDI standard (0-127)
                n.volume.velocity = min(max(int(n.volume.velocity * velocity_factor), 0), 127)
    return s

#### 2. Modify the velocity randomly to imitate the performed one

In [8]:
def randomize_velocity_in_measures(s, start_measure, end_measure, delta_range):
    """
    Randomly adjusts the velocity of each note within a specified range in a music stream,
    limited to a specific range of measures.

    Parameters:
        s (music21.stream.Stream): The music stream to modify.
        start_measure (int): The starting measure number.
        end_measure (int): The ending measure number.
        delta_range (int): The maximum change (up or down) that can be applied to the velocity.

    Returns:
        music21.stream.Stream: The modified music stream.
    """
    for measure_number in range(start_measure, end_measure + 1):
        measure = s.measure(measure_number)
        if measure:
            for n in measure.notes:  # Only adjust notes directly in the measure
                if n.volume.velocity is not None:  # Check if velocity is defined
                    change = random.randint(-delta_range, delta_range)  # Random change within the specified range
                    new_velocity = max(0, min(127, n.volume.velocity + change))  # Apply the change and clamp the result
                    n.volume.velocity = new_velocity  # Set the new velocity
                else:
                    n.volume.velocity = random.randint(64 - delta_range, 64 + delta_range)  # Default value if None
    return s

#### 3. Add the imitation of the pedals

In [9]:
def add_pedal_event(s, measure, beat, is_pedal_down, measure_offset):
    """
    Adds a pedal event to the stream at a specified beat within a measure.
    
    Parameters:
        s (music21.stream.Stream): The music stream to add the event to.
        measure (music21.stream.Measure): The measure where the event is added.
        beat (float): The beat within the measure to add the pedal event.
        is_pedal_down (bool): True if the pedal is pressed, False if released.
        measure_offset (float): The offset of the measure within the stream.
    """
    pedal_event = midi.ControlChange()
    pedal_event.channel = 1
    pedal_event.control = 64  # Sustain pedal control number
    pedal_event.value = 127 if is_pedal_down else 0
    pedal_event.time = measure_offset + (beat - 1) * measure.quarterLength / measure.timeSignature.beatCount
    
    s.events.append(pedal_event)
    return s

def apply_pedal_to_measures(s, start_measure, end_measure):
    """
    Applies the sustain pedal to specific measures in a 6/8 time signature stream.
    Pedal is pressed at 1/8 and released at 3/8, then pressed again at 4/8 and released at 6/8.
    
    Parameters:
        s (music21.stream.Stream): The music stream to modify.
        start_measure (int): The starting measure number (1-indexed).
        end_measure (int): The ending measure number (1-indexed).
    """
    for m in s.getElementsByClass('Measure'):
        if start_measure <= m.measureNumber <= end_measure:
            measure_offset = m.offset
            
            add_pedal_event(s, m, 1, True, measure_offset)   # Pedal down at 1/8
            add_pedal_event(s, m, 3, False, measure_offset)  # Pedal up at 3/8
            add_pedal_event(s, m, 4, True, measure_offset)   # Pedal down at 4/8
            add_pedal_event(s, m, 6, False, measure_offset)  # Pedal up at 6/8
    return s

#### 4. Imitate the trills

In [10]:
def apply_trill_to_hand_note(s, hand, measure_number, note_index, semitones, trill_speed, trill_duration):
    """
    Applies a custom trill effect to a specific note within a specified measure and specific hand part 
    in a music21 stream.

    Parameters:
        s (music21.stream.Stream): The music stream containing the measures and parts.
        hand (str): The hand part to apply the trill, 'left' or 'right'.
        measure_number (int): The measure number to find the note to trill.
        note_index (int): The index of the note within the specified measure to apply the trill.
        semitones (int): The number of semitones to transpose the original note by for the trill effect.
        trill_speed (float): The duration of each individual note in the trill, in quarter lengths.
        trill_duration (float): The total duration of the trill effect, in quarter lengths.
    """
    part = s.parts[0 if hand == 'right' else 1]
    target_measure = part.measure(measure_number)
    notes_in_measure = list(target_measure.flatten().notes)
    if note_index < len(notes_in_measure):
        trill_start_note = notes_in_measure[note_index]
        start_offset = trill_start_note.offset
        num_trills = int(trill_duration / trill_speed)
        trill_interval = interval.ChromaticInterval(semitones)
        trill_start_note.duration.quarterLength = trill_speed
        for i in range(1,num_trills):
            if i % 2 == 0:
                new_note = note.Note(trill_start_note.pitch,
                                     duration=duration.Duration(trill_speed))
            else:
                trill_pitch = trill_start_note.pitch.transpose(trill_interval)
                new_note = note.Note(trill_pitch,
                                     duration=duration.Duration(trill_speed))
            target_measure.insert(start_offset + i * trill_speed, new_note)
    return s

### Other functions 
#### 1. extract measures that is needed.

In [11]:
def extract_measures_and_save(input_stream, output_midi_path, start_measure, end_measure):
    """
    Extracts specified measures from a given music21 stream and saves them to a new MIDI file.

    :param input_stream: music21 stream object that contains the music data.
    :param output_midi_path: path where the new MIDI file will be saved.
    :param start_measure: the first measure to include in the extraction.
    :param end_measure: the last measure to include in the extraction.
    """
    new_stream = stream.Score()

    for part in input_stream.parts:
        # Extracts a specified range of bars
        extracted_measures = part.measures(start_measure, end_measure)
        
        new_part = stream.Part()
        for measure in extracted_measures:
            new_part.append(measure)
        new_stream.append(new_part)

    # save
    new_stream.write('midi', fp=output_midi_path)

In [12]:
# apply
mf = midi.MidiFile()
mf.open('corrected_midi_score.midi')
mf.read()
mf.close()

my_stream = converter.parse('corrected_midi_score.midi', format='midi', forceSource=True,
                            quantizePost=False, quarterLengthDivisors=(128, 48))

In [13]:
# adjust the duration of the notes on some beats
my_stream = change_duration_specific_beats_in_stream(my_stream, start_measure=43, end_measure=43, target_beats=[5,7], duration_factor=1.5)
my_stream = change_duration_specific_beats_in_stream(my_stream, start_measure=42, end_measure=42, target_beats=[6,7], duration_factor=1.2)


# 33-34 40-41 high pitch with higher intensity
my_stream = accentuate_highest_note_in_measure(my_stream,37,1.2)
my_stream = increase_volume_of_highest_note_in_triples(my_stream,40,41)
my_stream = increase_volume_of_higher_notes_in_track(my_stream, 45, 46)

# randomly adjust the velocity
my_stream = randomize_velocity_in_measures(my_stream, 31, 46, 2)

# paddles
my_stream = apply_pedal_to_measures(my_stream, 1, 70)

Measure 40 adjusted.
Measure 41 adjusted.
Measure 45 adjusted.
Measure 46 adjusted.


In [14]:
# add trills
# apply
my_stream = apply_trill_to_hand_note(my_stream, 'right', measure_number=43, note_index=-2, 
                         semitones=1, trill_speed=1/4, trill_duration=1)
my_stream = apply_trill_to_hand_note(my_stream, 'right', measure_number=43, note_index=-1, 
                         semitones=1, trill_speed=1/8, trill_duration=0.5)
my_stream = apply_trill_to_hand_note(my_stream, 'right', measure_number=44, note_index=0, 
                         semitones=2, trill_speed=1/4, trill_duration=1)
my_stream = apply_trill_to_hand_note(my_stream, 'right', measure_number=43, note_index=12, 
                         semitones=2, trill_speed=1/4, trill_duration=1)

In [15]:
# 创建一个新的 MIDI 文件并写入更改
my_stream.makeNotation() 
mf_out = midi.translate.music21ObjectToMidiFile(my_stream)
mf_out.open('whole_modified_midi_score.midi', 'wb')
mf_out.write()
mf_out.close()

In [16]:
output_midi_path = 'modified_midi_score.midi'
start_measure = 31
end_measure = 46
extract_measures_and_save(my_stream, output_midi_path, start_measure, end_measure)